# 국민청원 크롤링

In [4]:
petition_root_url = 'https://www1.president.go.kr/petitions'
for i in range(594316, 594321):
    URL = f'{petition_root_url}/{i}'
    print(URL)

https://www1.president.go.kr/petitions/594316
https://www1.president.go.kr/petitions/594317
https://www1.president.go.kr/petitions/594318
https://www1.president.go.kr/petitions/594319
https://www1.president.go.kr/petitions/594320


In [ ]:
import requests
URL = 'https://www1.president.go.kr/petitions/594316'
header = {'User-agent' : 'Mozila/2.0'}
response = requests.get(URL, headers=header)
html = response.text
html[3000:50000]

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

petition_root_url = 'https://www1.president.go.kr/petitions'
result = pd.DataFrame()
start = 594466
end = start + (60*100)
for i in range(start, end): 
    URL = f'{petition_root_url}/{i}'  
    header = {'User-agent' : 'Mozila/2.0'}
    response = requests.get(URL, headers=header)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.find('h3', class_='petitionsView_title')   # 국민청원 제목
    count = soup.find('span', class_='counter') # 청원 참여자수
    
    for content in soup.select('div.petitionsView_write > div.View_write'):   #본문
        content
        
    a = []
    for tag in soup.select('ul.petitionsView_info_list > li'):    # 메타정보 - 카테고리, 청원시작/끝날짜
        a.append(tag.contents[1])

    if len(a) != 0:
        df1 = pd.DataFrame({'start': [a[1]],
                         'end': [a[2]],
                         'category': [a[0]],
                         'count': [count.text],
                         'title': [title.text],
                         'content': [content.text.strip()[0: 13000]]
                        })
        result = pd.concat([result, df1])   # 청원별로 계속 concat
        result.index = np.arange(len(result))  # index 재부여        
    else:
        print(f'no a tag: {i}')
    
    if i !=0 and i % 60 == 0:                                                      
        print("Sleep 90 seconds. Count:" + str(i)           
              +",  Local Time:"+ time.strftime('%Y-%m-%d', time.localtime(time.time()))
              +" "+ time.strftime('%X', time.localtime(time.time()))
              +",  Data Length:"+ str(len(result)))        
        time.sleep(90) 
        
    if len(result) > 500:
        result.to_csv(f'./crawl/crawling_{i}.csv', index = False, encoding = 'utf-8-sig')        

        result = pd.DataFrame() # 리셋              
    